# template notebook to load data
- (option 1) load from S3 path
- (option 2) load from kedro

## Load data from S3 path
- make sure spark is installed

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window as w
import pandas as pd
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore")

spark = (
    SparkSession.builder
    .config("spark.driver.memory","100g")
    .config("spark.hadoop.fs.s3a.impl","org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.jars.packages","org.apache.hadoop:hadoop-aws:3.2.2")
    .config("fs.s3a.aws.credentials.provider","com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
    .config("spark.driver.maxResultSize",0)
    # .config('spark.hadoop.io.compression.codecs', 'nl.basjes.hadoop.io.compress.SplittableGzipCodec')
    .getOrCreate()
)
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)
spark.conf.set("spark.hadoop.io.compression.codecs", "org.apache.hadoop.io.compress.BZip2Codec")

:: loading settings :: url = jar:file:/home/jovyan/.conda/envs/python3/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-fe1e2c19-6ef1-46a7-a2aa-16e0f12386c5;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.2.2 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.563 in central
:: resolution report :: resolve 153ms :: artifacts dl 3ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.11.563 from central in [default]
	org.apache.hadoop#hadoop-aws;3.2.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	--------------------------------

In [2]:
!echo $team_bucket

820323602090-team-dbad373c-7e36-407b-8690-05a44b804f43


In [3]:
!aws s3 ls $team_bucket/data/raw/

                           PRE DIM_DIAGNOSIS/
                           PRE DIM_DRUG/
                           PRE DIM_HCPT/
                           PRE DX_DIAGNOSIS/
                           PRE MEDICAL_HEADERS/
                           PRE MEDICAL_SERVICE_LINES/
                           PRE PATIENT_ENROLLMENTS/
                           PRE PATIENT_SUMMARIES/
                           PRE PAYERS/
                           PRE PHARMACY/
                           PRE PROVIDER_SUMMARIES/
                           PRE VISIT_SUMMARIES/


In [5]:
!aws s3 ls $team_bucket/data/data_komodo/

                           PRE prm_diagnosis/
                           PRE prm_diagnosis_mk/
                           PRE prm_enrollment/
                           PRE prm_payers/
                           PRE prm_persons/
                           PRE prm_prescription/
                           PRE prm_prescription_1/
                           PRE prm_procedure/
                           PRE prm_providers/
                           PRE prm_visit/


In [7]:
s3_bucket_spark = "s3a://820323602090-team-dbad373c-7e36-407b-8690-05a44b804f43"
s3_bucket_pandas = "s3a://820323602090-team-dbad373c-7e36-407b-8690-05a44b804f43"

In [4]:
tmp_df = spark.read.parquet(s3_bucket_spark+"/data/raw/DX_DIAGNOSIS/")
tmp_df.columns

23/04/24 03:10:25 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


['DIAGNOSIS', 'DIAGNOSIS_CODE', 'SHORTDESCRIPTION']

In [5]:
tmp_df.show(5)

+---------+--------------+--------------------+
|DIAGNOSIS|DIAGNOSIS_CODE|    SHORTDESCRIPTION|
+---------+--------------+--------------------+
|      PSO|         L4050|Arthropathic psor...|
|      PSO|          L400|  Psoriasis vulgaris|
|      PSO|         L4051|Distal interphala...|
|      PSO|          L401|Generalized pustu...|
|      PSO|          L403|Pustulosis palmar...|
+---------+--------------+--------------------+
only showing top 5 rows



In [17]:
!aws s3 ls $team_bucket/data/hcp_universe/tremfya/

                           PRE negative/
                           PRE positive/


In [18]:
positive = spark.read.parquet(s3_bucket_spark+"/data/hcp_universe/tremfya/positive/")
positive.show(5)

+-------+------------+-----------+-----------------+
|   DRUG|PROVIDER_NPI| SPECIALTY1|       SPECIALTY2|
+-------+------------+-----------+-----------------+
|Tremfya|  1467492751|Dermatology|             null|
|Tremfya|  1821191099|Dermatology|             null|
|Tremfya|  1821210337|Dermatology|             null|
|Tremfya|  1821280298|Dermatology|Internal Medicine|
|Tremfya|  1821292301|Dermatology|             null|
+-------+------------+-----------+-----------------+
only showing top 5 rows



In [19]:
negative = spark.read.parquet(s3_bucket_spark+"/data/hcp_universe/tremfya/negative/")
negative.show(5)

+----------------+------------+-----------+----------+
|            DRUG|PROVIDER_NPI| SPECIALTY1|SPECIALTY2|
+----------------+------------+-----------+----------+
|Tremfya-negative|  1629141866|Dermatology|      null|
|Tremfya-negative|  1679507537|Dermatology|      null|
|Tremfya-negative|  1225233661|Dermatology|      null|
|Tremfya-negative|  1609815166|Dermatology|      null|
|Tremfya-negative|  1033287180|Dermatology|      null|
+----------------+------------+-----------+----------+
only showing top 5 rows



In [21]:
feature_and.count()

26185

In [19]:
feature_and = spark.read.parquet("../data/04_feature/patient_payer_months_features.parquet")
feature_and.show(3,truncate=False, vertical=True)

23/04/24 06:11:42 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


-RECORD 0-------------------------------------------------------
 is_train                               | 0                     
 PROVIDER_NPI                           | 1003145087            
 count_visits                           | 316                   
 unique_patients                        | 85                    
 unique_payers                          | 11                    
 avg_patient_age_months                 | 803.5770722744937     
 female_share                           | 0.5379746835443038    
 pat_from_NY_share                      | 0.00949367088607595   
 pat_from_CA_share                      | 0.0                   
 pat_from_FL_share                      | 0.9620253164556962    
 pat_from_TX_share                      | 0.0                   
 claim_month_1_share                    | 0.0949367088607595    
 claim_month_2_share                    | 0.06329113924050633   
 claim_month_3_share                    | 0.07278481012658228   
 claim_month_4_share     

In [20]:
feature_and.columns

['is_train',
 'PROVIDER_NPI',
 'count_visits',
 'unique_patients',
 'unique_payers',
 'avg_patient_age_months',
 'female_share',
 'pat_from_NY_share',
 'pat_from_CA_share',
 'pat_from_FL_share',
 'pat_from_TX_share',
 'claim_month_1_share',
 'claim_month_2_share',
 'claim_month_3_share',
 'claim_month_4_share',
 'claim_month_5_share',
 'claim_month_6_share',
 'claim_month_7_share',
 'claim_month_8_share',
 'claim_month_9_share',
 'claim_month_10_share',
 'claim_month_11_share',
 'payer_channel_Commercial_share',
 'payer_channel_Medicare_share',
 'payer_channel_Medicare Advantage_share',
 'payer_channel_Managed Medicaid_share',
 'payer_channel_Veterans Affairs_share',
 'payer_channel_Medicaid_share',
 'payer_channel_TRICARE_share',
 'count_visits_l3m',
 'unique_patients_l3m',
 'unique_payers_l3m',
 'avg_patient_age_months_l3m',
 'female_share_l3m',
 'count_visits_l6m',
 'unique_patients_l6m',
 'unique_payers_l6m',
 'avg_patient_age_months_l6m',
 'female_share_l6m']

# Load data from kedro
- please make sure this notebook lives within folder "/notebooks/" in order for "%reload_kedro" magic line to work
- alternatively, need to load kedro manually to access context

In [2]:
%reload_kedro

[04/19/23 20:25:45] INFO     Resolved project path as: /home/jovyan/personal/ ]8;id=56035;file:///home/jovyan/.conda/envs/test/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=618027;file:///home/jovyan/.conda/envs/test/lib/python3.8/site-packages/kedro/ipython/__init__.py#132\132]8;;\
                             zecai/codebase/life_sciences_hackathon.                         
                             To set a different path, run '%reload_kedro                     
                             <project_root>'                                                 

[04/19/23 20:25:46] WARNING  /home/jovyan/.conda/envs/test/lib/python3.8/site ]8;id=307279;file:///home/jovyan/.conda/envs/test/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=80492;file:///home/jovyan/.conda/envs/test/lib/python3.8/warnings.py#109\109]8;;\
                             -packages/kedro/framework/context/context.py:339                
                             : UserWarning: Credentials not found in your                    
                             Kedro project config.                                           
                             No files found in ['/home/jovyan/personal/zecai/                
                             codebase/life_sciences_hackathon/conf/base', '/h                
                             ome/jovyan/personal/zecai/codebase/life_sciences                
                             _hackathon/conf/local', '/home/jovyan/personal/z                
                             ecai/codebase/life_sciences_hackathon/src/pmpx_p                
                             kg/connectors/model_input/conf', '/home/jovyan/p                
                             ersonal/zecai/codebase/life_sciences_hackathon/s                
                             rc/pmpx_pkg/connectors/post_surrogate_tree/conf'                
                             , '/home/jovyan/personal/zecai/codebase/life_sci                
                             ences_hackathon/src/pmpx_pkg/connectors/data_kom                
                             odo/conf', '/home/jovyan/personal/zecai/codebase                
                             /life_sciences_hackathon/src/pmpx_pkg/connectors                
                             /segmentation/conf', '/home/jovyan/personal/zeca                
                             i/codebase/life_sciences_hackathon/src/pmpx_pkg/                
                             connectors/reporting_xai/conf', '/home/jovyan/pe                
                             rsonal/zecai/codebase/life_sciences_hackathon/sr                
                             c/pmpx_pkg/connectors/predictive_modeling/conf',                
                              '/home/jovyan/personal/zecai/codebase/life_scie                
                             nces_hackathon/src/pmpx_pkg/connectors/pxaa_core                
                             /conf'] matching the glob pattern(s):                           
                             ('credentials*', 'credentials*/**',                             
                             '**/credentials*')                                              
                               warn(f"Credentials not found in your Kedro                    
                             project config.\n{str(exc)}")                                   
                                                                                             

                    INFO     Kedro project pmpx_pkg                           ]8;id=165055;file:///home/jovyan/.conda/envs/test/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=96215;file:///home/jovyan/.conda/envs/test/lib/python3.8/site-packages/kedro/ipython/__init__.py#101\101]8;;\

                    INFO     Defined global variable 'context', 'session',    ]8;id=972717;file:///home/jovyan/.conda/envs/test/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=296543;file:///home/jovyan/.conda/envs/test/lib/python3.8/site-packages/kedro/ipython/__init__.py#102\102]8;;\
                             'catalog' and 'pipelines'                                       

                    INFO     Registered line magic 'run_viz'                  ]8;id=943013;file:///home/jovyan/.conda/envs/test/lib/python3.8/site-packages/kedro/ipython/__init__.py\__init__.py]8;;\:]8;id=586803;file:///home/jovyan/.conda/envs/test/lib/python3.8/site-packages/kedro/ipython/__init__.py#108\108]8;;\

In [3]:
tmp_df = catalog.load("data_komodo.prm_providers")
tmp_df.columns

[04/19/23 20:26:41] INFO     Loading data from                            ]8;id=702107;file:///home/jovyan/.conda/envs/test/lib/python3.8/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=670487;file:///home/jovyan/.conda/envs/test/lib/python3.8/site-packages/kedro/io/data_catalog.py#343\343]8;;\
                             'data_komodo.prm_providers'                                     
                             (SparkDataSet)...                                               

23/04/19 20:26:41 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


[
    'npi_id',
    'provider_speciality_1',
    'provider_speciality_2',
    'provider_state',
    'provider_type',
    'provider_hco_id',
    'provider_first_name',
    'provider_middle_name',
    'provider_last_name',
    'provider_deactivation_dt',
    'provider_reactivation_dt',
    'provider_process_dt',
    'source'
]